In [1]:
import os
MacOSPath = "/Users/grantseiter/Dropbox/github/Biden-Tax-Proposals/Tax-Calculator-3.0.0/"
MsWinPath = "C:\\Users\\grant.seiter\\Dropbox\\github\\Biden-Tax-Proposals\\Tax-Calculator-3.0.0\\"
os.chdir(MacOSPath)
workdir = os.getcwd()
print (workdir)
from taxcalc import *
from behresp import *
import pandas as pd
import numpy as np

/Users/grantseiter/Dropbox/github/Biden-Tax-Proposals/Tax-Calculator-3.0.0


In [2]:
# Elasticity Assumpitions
cg= {2021: -4.184,
     2022: -3.488,
     2023: -2.76,
     2024: -2.76,
     2025: -2.76,
     2026: -2.76,
     2027: -2.76,
     2028: -2.76,
     2029: -2.76,
     2030: -2.76
    }

In [3]:
def create_distribution_table(cyr, taxref, group):
    assert taxref in ['individual', 'individual_business']
    assert group in ['standard_income_bins', 'weighted_deciles']
    assert cyr in range(2021, 2031)
    if taxref=='individual':
        # baseline
        rec = Records()
        pol = Policy()
        calc_base=Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        #reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('../reforms/biden-iitax-reforms.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()
    else:
        # baseline
        rec = Records()
        pol = Policy()
        baseline={
            "Business_tax_expinc": {cyr: True}
        }
        pol.implement_reform(baseline)
        calc_base= Calculator(pol, rec)
        calc_base.advance_to_year(cyr)
        calc_base.calc_all()
        # reform
        rec = Records()
        pol = Policy()
        pol.implement_reform(Policy.read_json_reform('../reforms/biden-bustax-reforms.json'))
        calc_ref= Calculator(pol, rec)
        calc_ref.advance_to_year(cyr)
        calc_ref.calc_all()   
    # implement behavioral responses
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=True)
    # create difference table (1)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf1 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # create difference table (2) [only use the pc_aftertaxinc column from this distribution table]
    response_elasticities = {'sub': 0.25, 'cg': cg[cyr]}
    base_df, ref_df = response(calc_base, calc_ref, response_elasticities, chg_inc=False)
    vdf1 = base_df[DIFF_VARIABLES]
    vdf2 = ref_df[DIFF_VARIABLES]
    distdf2 = create_difference_table(vdf1, vdf2, groupby=group, tax_to_diff='combined', pop_quantiles=False)
    # append pc_aftertaxinc from dataframe 2 and delete unused columns
    df = distdf1.assign(pch_chg_inc = distdf2['pc_aftertaxinc'])
    table = df.drop(['ubi', 'benefit_cost_total', 'benefit_value_total', 'pc_aftertaxinc', 'count', 'tax_cut',
                   'perc_cut', 'tax_inc', 'perc_inc', 'tot_change'], axis=1)
    # final table, by standard income bin
    return table

#### TABLE 3: TOTAL BURDEN: INDIVIDUAL AND BUSINESS REFORMS

In [4]:
create_distribution_table(cyr=2021, taxref='individual_business', group='weighted_deciles')

,mean,share_of_change,pch_chg_inc
0-10n,4589.7,7.2,3.7
0-10z,-80.0,-0.3,61.9
0-10p,-310.7,-4.8,11.3
10-20,-595.2,-12.5,5.7
20-30,-888.1,-18.7,4.9
30-40,-756.5,-15.9,3.0
40-50,-598.7,-12.6,1.8
50-60,-474.1,-10.0,1.1
60-70,-392.0,-8.3,0.7
70-80,-446.3,-9.4,0.6


In [5]:
create_distribution_table(cyr=2030, taxref='individual_business', group='weighted_deciles')

,mean,share_of_change,pch_chg_inc
0-10n,13942.0,5.4,9.3
0-10z,16.1,0.0,-9.0
0-10p,23.2,0.1,-0.6
10-20,24.5,0.1,-0.2
20-30,30.3,0.2,-0.1
30-40,36.0,0.2,-0.1
40-50,40.0,0.2,-0.1
50-60,49.4,0.3,-0.1
60-70,101.5,0.6,-0.1
70-80,194.0,1.1,-0.2



#### INDIVIDUAL REFORMS

In [6]:
create_distribution_table(cyr=2021, taxref='individual', group='weighted_deciles')

,mean,share_of_change,pch_chg_inc
0-10n,12178.8,-16.4,9.2
0-10z,-80.0,0.3,61.9
0-10p,-315.2,4.2,11.6
10-20,-622.0,11.3,6.1
20-30,-937.9,17.0,5.3
30-40,-841.8,15.3,3.3
40-50,-735.9,13.3,2.2
50-60,-715.4,13.0,1.7
60-70,-712.2,12.9,1.3
70-80,-920.3,16.7,1.2


In [7]:
create_distribution_table(cyr=2030, taxref='individual', group='weighted_deciles')

,mean,share_of_change,pch_chg_inc
0-10n,22527.7,26.3,14.7
0-10z,16.1,0.0,-9.0
0-10p,16.8,0.2,-0.5
10-20,-8.7,-0.1,0.1
20-30,-35.7,-0.6,0.2
30-40,-72.7,-1.2,0.2
40-50,-139.6,-2.3,0.3
50-60,-259.0,-4.3,0.5
60-70,-331.7,-5.5,0.5
70-80,-433.1,-7.2,0.5
